Bad Character Heuristic (Bad Character table)

In [43]:
def bad_character_dictionary(pattern):
    """
    Constructs a dictionary that contains the rightmost position of each character in the pattern.

    Args:
        pattern (str): The pattern string.

    Returns:
        dict: A dictionary where the keys are characters in the pattern and the values are their rightmost positions.
    """
    rightmost = {c: -1 for c in pattern}
    for i, c in enumerate(pattern):
        rightmost[c] = i
    return rightmost


def bad_character_heuristic(pattern, text, rightmost):
    """
    Applies the bad character heuristic to find the shift distance for pattern matching.

    Args:
        pattern (str): The pattern to be matched.
        text (str): The text in which the pattern is to be searched.
        rightmost (dict): A dictionary containing the rightmost occurrence of each character in the pattern.

    Returns:
        int: The shift distance for pattern matching.
    """
    # 从右向左比较 pattern 和 text
    patternLength = len(pattern)
    for i in range(patternLength - 1, -1, -1):
        if pattern[i] != text[i]:
            # 不匹配的情况
            bad_char = text[i]
            if bad_char in rightmost:
                # 如果 text 中的不匹配字符存在于 pattern 中
                # 计算移动距离: 目标位置 - 当前位置
                shift = max(1, i - rightmost[bad_char])
            else:
                # 如果 text 中的不匹配字符不在 pattern 中
                # 移动距离：整个 pattern 右移至不匹配字符后一位
                shift = i + 1
            return shift

    # 如果字符串完全匹配
    return 0

Good Suffix heuristic

In [44]:
def good_suffix_shift(pattern, enable_print=False):
    """
    Calculates the good suffix shift array for the given pattern.

    Args:
        pattern (str): The pattern string.
        enable_print (bool, optional): Whether to enable print statements for debugging. Defaults to False.

    Returns:
        list: The good suffix shift array.

    Raises:
        None

    Example:
        pattern = "abc"
        good_suffix_shift(pattern) => [3, 3, 3]
    """
    pattern_length = len(pattern)
    suffix = [0] * (pattern_length * 2)
    goodSuffix = [pattern_length] * pattern_length  # 初始化gs数组，假设在没有好后缀的情况下，移动整个模式长度

    # 计算suffix数组
    if enable_print:
        print(f"Initial suffix array: {suffix}")
        print(f"Initial goodSuffix array: {goodSuffix}")

    # 设置模式串最后一个字符的suffix值为模式串的长度，因为整个模式串都是自身的后缀
    suffix[pattern_length - 1] = pattern_length
    # 初始化f为模式串的最后一个索引
    f = pattern_length - 1

    if enable_print:
        print(f"Suffix array after initialization: {suffix}")

    print('\n')

    # 从模式串的倒数第二个字符开始向前遍历
    for i in range(pattern_length - 2, -1, -1):
        # 如果当前索引i大于f且通过之前的计算得到的suffix长度小于从i到f的距离
        # 这表明当前的后缀可以通过之前计算的更长的后缀来简化
        if i > f and suffix[i + pattern_length - 1 - f] < i - f:
            suffix[i] = suffix[i + pattern_length - 1 - f]
            if enable_print:
                print(f"i = {i}: suffix[{i}] can be reused from suffix[{i + pattern_length - 1 - f}] = {suffix[i]}")
        else:
            # 如果当前索引i不满足上述条件，重新设置f为i，重新计算最长公共后缀
            if i < f:
                f = i
            # 从i开始向左对比，计算从i开始的后缀与模式串的后缀的最长公共部分
            # f与模式串的后缀的比较位置同步向左移动
            k = f
            while k >= 0 and pattern[k] == pattern[k + pattern_length - 1 - i]:
                if enable_print:
                    print(f"i = {i}: suffix[{i}] is comparing pattern[{k}] ('{pattern[k]}') with pattern[{k + pattern_length - 1 - i}] ('{pattern[k + pattern_length -1 - i] if k + pattern_length -1 - i < pattern_length else 'OutOfBound'})")
                k -= 1
            # 最长公共后缀的长度为i-f
            suffix[i] = i - k
            if enable_print:
                if k>=0:
                    print(f"i = {i}: suffix[{i}] is calculated as {suffix[i]}, pattern[{k}] ('{pattern[k]}') != pattern[{k + pattern_length - 1 - i}] ('{pattern[k + pattern_length -1 - i] if k + pattern_length -1 - i < pattern_length else 'OutOfBound'})")
                else:
                    #匹配到头
                    #print It is last character of pattern. Thus suffix is equal i+1
                    print(f'It is last character of pattern. Thus suffix is equal {i+1}')
        if enable_print:
            print(f"Suffix array at step {pattern_length - 1 - i}: {suffix} \n")

    # 计算好后缀移动数组gs
    j = 0
    for i in range(pattern_length - 1, -1, -1):
        if suffix[i] == i + 1:  # 全后缀匹配的情况
            while j < pattern_length - 1 - i:
                if goodSuffix[j] == pattern_length:  # 只有当gs[j]未被设置时才更新
                    goodSuffix[j] = pattern_length - 1 - i
                    if enable_print:
                        print(f"goodSuffix[{j}] is set to {goodSuffix[j]} because suffix[{i}] = {suffix[i]} indicates a full suffix match")
                j += 1
        if enable_print:
            print(f"GoodSuffix array at step {pattern_length - 1 - i}: {goodSuffix} \n")

    # 其他情况，根据suffix值设置gs值
    for i in range(pattern_length - 1):
        goodSuffix[pattern_length - 1 - suffix[i]] = pattern_length - 1 - i
        if enable_print:
            print(f"GoodSuffix array after final adjustment at step {i + 1}: {goodSuffix}")

    return goodSuffix


def good_suffix_heuristic(pattern, text, goodSuffix):
    """
    Applies the good suffix heuristic to find the shift value for a pattern in a text.

    Args:
        pattern (str): The pattern to search for in the text.
        text (str): The text to search in.
        goodSuffix (list): A list containing the shift values for each suffix of the pattern.

    Returns:
        int: The shift value for the pattern in the text.
    """

    pattern_length = len(pattern)

    j = pattern_length - 1
    while j >= 0 and pattern[j] == text[j]:
        j -= 1
    
    if j < 0:
        return 0
    
    return goodSuffix[j]

In [45]:
def boyer_moore(pattern, text, enable_print=False, enable_suffix_print=False):
    """
    Performs the Boyer-Moore string matching algorithm to find all occurrences of a pattern in a text.

    Args:
        pattern (str): The pattern to search for.
        text (str): The text to search in.
        enable_print (bool, optional): Whether to enable visual display of the alignment. Defaults to False.
        enable_suffix_print (bool, optional): Whether to enable visual display of the suffix heuristic. Defaults to False.

    Returns:
        list: A list of indices where the pattern is found in the text.
    """
    pattern_length = len(pattern)
    text_length = len(text)

    goodSuffix = good_suffix_shift(pattern, enable_suffix_print)
    badCharacter = bad_character_dictionary(pattern)

    
    if text_length < pattern_length:
        return []
    
    matches = []  # used to store all matching positions
    i = 0  # initialize the shift
    
    while i <= text_length - pattern_length:
        # get the shift based on the bad character heuristic
        shift_bad_char = bad_character_heuristic(pattern, text[i:i + pattern_length], badCharacter)
        
        # get the shift based on the good suffix heuristic
        shift_good_suffix = good_suffix_heuristic(pattern, text[i:i + pattern_length], goodSuffix)
        
        # visualize the current alignment
        if enable_print:
            print(text)
            print(' ' * i + pattern)
        
        # if there is a complete match, record the matching position
        if shift_bad_char == 0 and shift_good_suffix == 0:
            matches.append(i)
            if enable_print:
                print(f"Pattern found at index {i}.")
                print('\n' )
            i += pattern_length  # move the pattern, skip the current matching position
        else:
            # take the maximum shift from the two heuristics
            shift = max(shift_bad_char, shift_good_suffix)
            if enable_print:
                print(f"Pattern not found at index {i}. Bad char shift: {shift_bad_char}, Good suffix shift: {shift_good_suffix}, Shifting by: {shift}")
            i += shift
            if enable_print:
                print('\n')
    
    return matches

In [46]:
text = "AXAAXAAxAAAXAXXAAA"
pattern = "AXA"

result = boyer_moore(pattern, text, enable_print=True,enable_suffix_print=False)
print(f"Pattern found at indices: {result}")



AXAAXAAxAAAXAXXAAA
AXA
Pattern found at index 0.


AXAAXAAxAAAXAXXAAA
   AXA
Pattern found at index 3.


AXAAXAAxAAAXAXXAAA
      AXA
Pattern not found at index 6. Bad char shift: 2, Good suffix shift: 2, Shifting by: 2


AXAAXAAxAAAXAXXAAA
        AXA
Pattern not found at index 8. Bad char shift: 1, Good suffix shift: 2, Shifting by: 2


AXAAXAAxAAAXAXXAAA
          AXA
Pattern found at index 10.


AXAAXAAxAAAXAXXAAA
             AXA
Pattern not found at index 13. Bad char shift: 1, Good suffix shift: 2, Shifting by: 2


AXAAXAAxAAAXAXXAAA
               AXA
Pattern not found at index 15. Bad char shift: 1, Good suffix shift: 2, Shifting by: 2


Pattern found at indices: [0, 3, 10]


Knuth–Morris–Pratt algorithm

In [75]:
def compute_lps(pattern):
    """
    Compute the Longest Prefix Suffix (LPS) array for the pattern.
    """
    length = 0  # Length of the previous longest prefix suffix
    lps = [0] * len(pattern)  # Initialize the LPS array
    i = 1

    print("Computing LPS array:")
    while i < len(pattern):
        if pattern[i] == pattern[length]:
            length += 1
            lps[i] = length
            i += 1
            print(f"Match: i={i-1}, length={length}, lps={lps}")
        else:
            if length != 0:
                length = lps[length - 1]
                print(f"Mismatch and length != 0: i={i}, length={length}, lps={lps}")
            else:
                lps[i] = 0
                i += 1
                print(f"Mismatch and length == 0: i={i-1}, length={length}, lps={lps}")

    return lps

def kmp_search(text, pattern):
    """
    KMP string matching algorithm to find all occurrences of a pattern in a text.
    """
    textLength = len(text)
    patternLength = len(pattern)
    lps = compute_lps(pattern)
    textIndex = 0  # Index for text
    patternIndex = 0  # Index for pattern
    positions = []

    print("\nStarting search:")
    while textIndex < textLength:
        if pattern[patternIndex] == text[textIndex]:
            textIndex += 1
            patternIndex += 1
        
        # Display current alignment with arrows
        print(f"\n{text}")
        print(' ' * (textIndex - patternIndex) + pattern)
        print(' ' * (textIndex-1) + '^' + ' ' * (textLength - textIndex - 1) + " (Text pointer)")
        print(' ' * (textIndex - patternIndex) + '^' + ' ' * (patternLength - (textIndex - patternIndex) - 1) + " (Pattern pointer)")
        
        if patternIndex == patternLength:
            positions.append(textIndex - patternIndex)
            print(f"Pattern found at index {textIndex-patternIndex}.")
            patternIndex = lps[patternIndex - 1]
            print(f"Next step: Shift the pattern to align with LPS[{patternIndex}].")
        
        elif textIndex < textLength and pattern[patternIndex] != text[textIndex]:
            if patternIndex != 0:
                patternIndex = lps[patternIndex - 1]
                print(f"Mismatch at textIndex={textIndex}, patternIndex={patternIndex}. Pattern shifted to the right based on LPS.")
            else:
                textIndex += 1
                print(f"Mismatch at textIndex={textIndex-1}, patternIndex={patternIndex}. Moving to the next character in the text.")

    return positions

# Example usage
text = "AXAAXAAxAAAXAXXAAA"
pattern = "AXA"
result = kmp_search(text, pattern)
print(f"\nLPS array: {compute_lps(pattern)}")
print(f"\nPattern found at indices: {result}")


Computing LPS array:
Mismatch and length == 0: i=1, length=0, lps=[0, 0, 0]
Match: i=2, length=1, lps=[0, 0, 1]

Starting search:

AXAAXAAxAAAXAXXAAA
AXA
^                 (Text pointer)
^   (Pattern pointer)

AXAAXAAxAAAXAXXAAA
AXA
 ^                (Text pointer)
^   (Pattern pointer)

AXAAXAAxAAAXAXXAAA
AXA
  ^               (Text pointer)
^   (Pattern pointer)
Pattern found at index 0.
Next step: Shift the pattern to align with LPS[1].

AXAAXAAxAAAXAXXAAA
  AXA
  ^               (Text pointer)
  ^ (Pattern pointer)
Mismatch at textIndex=3, patternIndex=0. Pattern shifted to the right based on LPS.

AXAAXAAxAAAXAXXAAA
   AXA
   ^              (Text pointer)
   ^ (Pattern pointer)

AXAAXAAxAAAXAXXAAA
   AXA
    ^             (Text pointer)
   ^ (Pattern pointer)

AXAAXAAxAAAXAXXAAA
   AXA
     ^            (Text pointer)
   ^ (Pattern pointer)
Pattern found at index 3.
Next step: Shift the pattern to align with LPS[1].

AXAAXAAxAAAXAXXAAA
     AXA
     ^            (Text pointer)
   

### References

1. [Boyer Moore Algorithm for Pattern Searching](https://www.geeksforgeeks.org/boyer-moore-algorithm-for-pattern-searching/)
2. [Boyer Moore Algorithm - Good Suffix Heuristic](https://www.geeksforgeeks.org/boyer-moore-algorithm-good-suffix-heuristic/)